In [1]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import Sequential,MaxPool2d,Linear,ReLU
from functions_pytorch import *
from pytorch_util import *
from pytorch_models import ConvBatchRelu
from albumentations_helper import create_transform
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
import pickle
import numpy as np
import pretrainedmodels
from functools import partial

In [2]:
## configs ##
color = True
#shapes = (3,224,224)
HalfBatch = 8
outDim = 500
distanceFun = l2_distance
distanceFun_np = l2_distance_np
TTASize = 4
pct = 0.1

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

Set up dataloader

In [4]:
aug = Compose([RandomContrast(p=0.5),RandomBrightness(p=0.5),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=25,scale_limit=0.05,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

In [5]:
gen_train = TripletGenerator(Ids_train,newWhale_train,transform)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test)        
#gen_train = FunctionWrapOverDataset(gen_train,numpy2torch)
#gen_val = FunctionWrapOverDataset(gen_val,numpy2torch)
train_dl= DataLoader(gen_train,HalfBatch,True,num_workers=3,drop_last=True)
valid_dl = DataLoader(gen_val,HalfBatch,False,num_workers=3,drop_last=True)

Set up training

In [6]:
# convNet = Sequential(ConvBatchRelu(3,8,5,stride=2),\
#                      ConvBatchRelu(8,16,5,stride=2),\
#                      ConvBatchRelu(16,32,5,stride=2),\
#                      MaxPool2d(2))

# fcNet = Sequential(Linear(4608,32),ReLU(True),Linear(32,6))

# with torch.no_grad():
#     fcNet[2].weight.data.zero_()
#     fcNet[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

# stn = SpatialTransformerNet(convNet,fcNet)

In [7]:
#base = pretrainedmodels.resnet50()
# base = pretrainedmodels.densenet121()
# base = fine_tune_pretrainedmodels(base, outDim)
# model = Sequential(stn,base).to('cuda:0')

In [6]:
model = pretrainedmodels.densenet121()
model = fine_tune_pretrainedmodels(model, outDim).to('cuda:0')

In [8]:
loss_func = loss_func_generator_softmax(HalfBatch,distanceFun)

In [9]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [10]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.9363746187210084, val_loss:0.6307057738304138
epoch:1, train_loss:0.6406181312263012, val_loss:0.5564051866531372
epoch:2, train_loss:0.5792698266804218, val_loss:0.5632326602935791
epoch:3, train_loss:0.5105037847280502, val_loss:0.5851256847381592
epoch:4, train_loss:0.48829534257650375, val_loss:0.547940731048584
Training completed in 149.40972089767456s


Fine-tune previous layers

In [ ]:
# for ResNet50
# set_requires_grad([model[1].layer4,model[1].layer3,model[1].layer2],True)
# opt = Adam([
#             {"params": model[1].layer4.parameters(), "lr": 1e-5},
#             {"params": model[1].layer3.parameters(), "lr": 1e-6},
#             {"params": model[1].layer2.parameters(), "lr": 5e-7},
#             {"params": model[1].last_linear.parameters(), "lr": 2e-5},
#             {"params": model[0].parameters(), "lr": 2e-5}
#             ])

In [11]:
# for DenseNet121
name_list = ['denseblock3','transition3','denseblock4','norm5']
set_requires_grad_paraList(gather_parameter_nameList(name_list,model),True)

In [12]:
opt = Adam([
            {"params": gather_parameter_nameList(['denseblock4','norm5'],model), "lr": 1e-4},
            {"params": gather_parameter_nameList(['denseblock3','transition3'],model), "lr": 1e-5},
            {"params": gather_parameter_nameList(['denseblock2','transition2'],model), "lr": 1e-6},
            {"params": model.last_linear.parameters(), "lr": 2e-4}
            ])

In [13]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.40156934076547623, val_loss:0.4499461054801941
epoch:1, train_loss:0.30486992998719215, val_loss:0.43556976318359375
epoch:2, train_loss:0.24957322633862494, val_loss:0.3282415568828583
epoch:3, train_loss:0.19049111304283142, val_loss:0.2901590168476105
epoch:4, train_loss:0.17467468890547752, val_loss:0.273586630821228
Training completed in 258.39065957069397s


In [17]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk1_softmax')

In [ ]:
# MAP -> 0.2452690166975881

HalfBatch*2

In [25]:
gen_train = TripletGenerator(Ids_train,newWhale_train,transform)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test)        
#gen_train = FunctionWrapOverDataset(gen_train,numpy2torch)
#gen_val = FunctionWrapOverDataset(gen_val,numpy2torch)
train_dl= DataLoader(gen_train,HalfBatch*2,True,num_workers=3,drop_last=True)
valid_dl = DataLoader(gen_val,HalfBatch*2,False,num_workers=3,drop_last=True)

In [27]:
loss_func = loss_func_generator_softmax(HalfBatch*2,distanceFun)

In [28]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.2942174540247577, val_loss:0.34849482774734497
epoch:1, train_loss:0.2525159327361064, val_loss:0.34434640407562256
epoch:2, train_loss:0.23249604321347597, val_loss:0.3346673548221588
epoch:3, train_loss:0.23243815917521715, val_loss:0.27159592509269714
epoch:4, train_loss:0.1989034809446774, val_loss:0.2889777719974518
Training completed in 192.50866723060608s


In [29]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk2_softmax')

In [ ]:
# MAP -> 0.37068645640074216

HalfBatch*3

In [37]:
gen_train = TripletGenerator(Ids_train,newWhale_train,transform)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test)        
#gen_train = FunctionWrapOverDataset(gen_train,numpy2torch)
#gen_val = FunctionWrapOverDataset(gen_val,numpy2torch)
train_dl= DataLoader(gen_train,HalfBatch*3,True,num_workers=3,drop_last=True)
valid_dl = DataLoader(gen_val,HalfBatch*3,False,num_workers=3,drop_last=True)

In [38]:
loss_func = loss_func_generator_softmax(HalfBatch*3,distanceFun)

In [39]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl,lossBest=None)

epoch:0, train_loss:0.2527885726079918, val_loss:0.33201363682746887
epoch:1, train_loss:0.23088736241110241, val_loss:0.3115643858909607
epoch:2, train_loss:0.21344923155489737, val_loss:0.29867690801620483
epoch:3, train_loss:0.21914900266207182, val_loss:0.29988279938697815
epoch:4, train_loss:0.1989872565139264, val_loss:0.321312814950943
Training completed in 174.56020164489746s


In [47]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, 'Models/chk3_softmax')

In [ ]:
# MAP -> 0.4474953617810761

Validation

In [40]:
pred_train_gen = PredictGenerator(Ids_train.Imgs.tolist(),transform_test,TTASize, True)
pred_val_gen = PredictGenerator(Ids_val.Imgs.tolist(),transform_test,TTASize, True)

In [41]:
feature_train = predict(model,pred_train_gen)
feature_val = predict(model,pred_val_gen)

In [42]:
feature_train = feature_train.reshape(len(pred_train_gen),TTASize,outDim)
feature_val = feature_val.reshape(len(pred_val_gen),TTASize,outDim)

In [43]:
aggFun = partial(np.quantile,q=pct,axis=(1,2))

In [44]:
predicts = loop_distance(feature_train,feature_val,distanceFun_np,aggFun,returnValue=False,k=5)

In [45]:
mapping_dict = dict(zip(Ids_train.Id.values,Ids_train.index.values))
labels = Ids_val.Id.map(mapping_dict)

In [46]:
MAP(labels,predicts)

0.4474953617810761